In [22]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pims
import pickle
import skimage.measure
from collections import defaultdict

In [2]:
home_path = '/home/idies/workspace/codex/PD1_polarization/'
cell_data = pd.read_csv(home_path+'reg5_mouse89_outlines_and_expression.csv')
shared_dir = "Storage/bmyury/shared_scripts_and_files/"

In [3]:
channelNames = list(pd.read_csv(home_path+'channelNames.txt',header = None)[0].values)

In [4]:
cell_data.head()

,CellID,tilename,Xroi,Yroi,Xtile,Ytile,ClusterID,CD62L,Siglec-H,CD86,...,X_withinTile,Y_withinTile,Z,size,cyc20_ch2blnk,cyc20_ch3blnk,cyc20_ch4blnk,cyc21_ch2blnk,cyc21_ch3blnk,cyc21_ch4blnk
0,0-1201-4,reg005_X01_Y02,2_2_2_2_2_2_2_2_2_2_2_1_1_1_1_1_1_1_1_1_1_1_0_...,478_479_480_481_482_483_484_485_486_487_488_47...,1,2,3262,477.369232,46.163620,123.079987,...,0,481,4,566,0.086064,0.000000,0.000000,0.511932,0.000000,0.000000
1,0-1216-4,reg005_X01_Y02,2_2_2_2_2_2_2_2_2_2_2_2_2_2_1_2_1_1_1_1_1_1_1_...,489_490_491_492_493_494_495_496_497_498_499_50...,1,2,3262,0.000000,0.000000,117.690712,...,0,496,4,585,0.000000,3.202495,4.067975,0.000000,12.229112,21.761768
2,0-123-4,reg005_X01_Y01,0_0_0_0_0_1_0_1_0_1_0_1_0_1_0_1_0_1_0_1_0_1_0_...,104_105_106_107_108_104_109_105_110_106_111_10...,1,1,3223,0.000000,0.000000,0.000000,...,0,123,4,4086,36.213600,5.332117,1.150246,90.962128,16.062859,4.204434
3,0-1268-3,reg005_X01_Y03,0_0_0_0_0_0_0_0_0_0_0_1_0_0_1_0_1_0_0_1_0_1_0_...,26_27_28_29_30_31_32_33_34_35_36_26_37_38_27_3...,1,3,3262,383.789124,62.661911,265.210876,...,0,44,3,2519,0.000000,2.596656,4.500194,0.000000,11.984420,16.527153
4,0-1285-3,reg005_X01_Y03,0_0_0_0_0_0_0_0_0_1_1_1_1_1_1_1_1_1_1_2_2_2_2_...,57_58_59_60_62_63_64_65_66_58_59_60_61_62_63_6...,1,3,3246,0.000000,0.000000,8.862731,...,0,61,3,385,1.368026,0.000000,0.000000,3.603063,0.000000,0.000000


In [5]:
def get_channel_index(chan,num_zs=7,z_indices = range(7)):
    cycle = chan//4
    channel = chan%4
    index = [cycle*4*num_zs+(z*4)+channel for z in z_indices]
    return index  

    
    
def read_cells(tile_name,cell_data):
    '''
    extract cells of given tile.  
    
    tile_name:  string of tile name (not including '.tif' e.g: reg005_X01_Y01)
    cell_data:  dataframe of cell regions, expression info and other meta data
    
    returns subset of cell_data corresponding to a tile and dict with keys as CellID and values as np array of region pixels 
    *note values are in x,y format (not rows, cols)
    '''
    tile_cells = cell_data[cell_data['tilename']==tile_name]
    roisX = np.core.defchararray.split(tile_cells['Xroi'].values.astype(str),sep = '_')
    roisY = np.core.defchararray.split(tile_cells['Yroi'].values.astype(str),sep = '_')
    roi = {i:np.array([x,y]).astype(int) for (i,x,y) in zip(tile_cells['CellID'].values,roisX,roisY) if (len(x)>1 and len(y)>1)} 
    return tile_cells,roi

def read_tile_image(tile_name,channelNames,markers,zs,tile_direc ):
    '''
    extract desired channels and zs from raw image
    
    tile_name:  string of tile name (not including '.tif' e.g: reg005_X01_Y01)
    channelNames:  list of proteins in order found in image (not including Zs)
    markers:    list of proteins to view  e.g: PD1,CD3 etc 
    zs:         list of z coordinates to extract
    tile_direc:  directory where tile data is saved
    
    *note raw image is flattened from following shape (num_cycles,num_zs,num_channels)
    
    returns: numpy array in following shape (num_markers,num_zs,nrows,ncols)
    '''
    indices = [get_channel_index(channelNames.index(marker),z_indices = zs) for marker in markers]
    indices = [k for i in indices for k in i]
    path = '{}/{}.tif'.format(tile_direc,tile_name)
    s = pims.TiffStack(path)
    img = np.array(s[indices]).reshape(len(markers),len(zs),*s.frame_shape)
    return img

 
def match_tile_and_cells(tile_name,cell_data,channelNames,markers,tile_direc = home_path+'mouse_number_89_reg005_tiles'):
    '''
    wrapper function to get imge and cells of a given tile with desired markers and zs 
    
    tile_name:  string of tile name (not including '.tif' e.g: reg005_X01_Y01)
    cell_data:  dataframe of cell regions, expression info and other meta data
    channelNames:  list of proteins in order found in image (not including Zs)
    markers:    list of proteins to view  e.g: PD1,CD3 etc
    tile_direc:  directory where tile data is saved
    
    returns: np.array(num_markers,num_zs,nrows,ncols),dataframe (subset of cell_data), dict cellid: cell pixels (x,y))
    
    '''
    img = read_tile_image(tile_name,channelNames,markers,zs,tile_direc)
    tile_cells,roi = read_cells(tile_name,cell_data)
    return img,tile_cells,roi
    
    
    
    

# Segmentation masks

In [6]:
tile_ids = cell_data["tilename"].unique()
tile_sizes = {
    ti: pims.TiffStack(home_path + "{}/{}.tif".format("mouse_number_89_reg005_tiles", ti)).frame_shape for ti in tile_ids
}


In [34]:
cell_data["cell_ix"] = np.arange(cell_data.shape[0]) + 1
cell_id_ix_map = {
    x[0]: x[1] for x in zip(cell_data["CellID"], cell_data["cell_ix"])
}
cell_ix_id_map = {
    v: k for k, v in cell_id_ix_map.items()
}

In [30]:
segmentation_masks = {}
cell_data_grouped = cell_data.groupby(["Z", "tilename"])
for group, cd in cell_data_grouped:
    ti = group[1]
    ts = tile_sizes[ti]
    z = group[0]
    print(ti)
    _, roi = read_cells(ti, cd)
    sm = np.full(ts, 0, dtype=np.uint32)
    for ci, cc in roi.items():
        sm[cc[1], cc[0]] = cell_id_ix_map[ci]
    segmentation_masks[(ti, z)] = sm

reg005_X01_Y06
reg005_X01_Y06
reg005_X04_Y01
reg005_X07_Y07
reg005_X07_Y09
reg005_X01_Y06
reg005_X01_Y09
reg005_X02_Y01
reg005_X02_Y03
reg005_X02_Y06
reg005_X03_Y01
reg005_X03_Y05
reg005_X04_Y05
reg005_X04_Y07
reg005_X04_Y09
reg005_X05_Y04
reg005_X05_Y05
reg005_X05_Y06
reg005_X06_Y05
reg005_X06_Y07
reg005_X06_Y09
reg005_X07_Y01
reg005_X07_Y02
reg005_X07_Y03
reg005_X07_Y04
reg005_X07_Y05
reg005_X07_Y06
reg005_X07_Y09
reg005_X01_Y01
reg005_X01_Y02
reg005_X01_Y03
reg005_X01_Y04
reg005_X01_Y05
reg005_X01_Y06
reg005_X01_Y07
reg005_X01_Y09
reg005_X02_Y01
reg005_X02_Y02
reg005_X02_Y03
reg005_X02_Y05
reg005_X02_Y06
reg005_X02_Y07
reg005_X02_Y08
reg005_X02_Y09
reg005_X03_Y01
reg005_X03_Y03
reg005_X03_Y04
reg005_X03_Y05
reg005_X03_Y07
reg005_X03_Y09
reg005_X04_Y01
reg005_X04_Y03
reg005_X04_Y04
reg005_X04_Y05
reg005_X04_Y06
reg005_X04_Y07
reg005_X04_Y09
reg005_X05_Y01
reg005_X05_Y02
reg005_X05_Y03
reg005_X05_Y04
reg005_X05_Y05
reg005_X05_Y06
reg005_X05_Y07
reg005_X05_Y08
reg005_X05_Y09
reg005_X06

In [31]:
segmentation_masks[("reg005_X06_Y03", 3)]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

In [32]:
with open(shared_dir + "/segmentation_masks.pkl", "wb") as f:
    pickle.dump(segmentation_masks, f)

In [8]:
# with open(shared_dir + "/segmentation_masks.pkl", "rb") as f:
#     segmentation_masks = pickle.load(f)

# Region props

In [9]:
selected_channels = [x for x in channelNames if "blnk" not in x]

In [10]:
selected_channels

['DNA_1',
 'CD62L',
 'Siglec-H',
 'FOXP3',
 'DNA_2',
 'CD86',
 'TCRgd',
 'NKp46',
 'DNA_3',
 'CD25',
 'CD69',
 'CXCR5',
 'DNA_4',
 'Ki67',
 'DNA_5',
 'CD90',
 'PNAd',
 'TYRP1',
 'DNA_6',
 'CD11b',
 'podoplanin',
 'CD11c',
 'DNA_7',
 'Ter119',
 'F4/80',
 'CD106',
 'DNA_8',
 'CD5',
 'H-2Db',
 'DNA_9',
 'CD71',
 'H-2Kb',
 'CD169',
 'DNA_10',
 'CD44',
 'CD8a',
 'PD1',
 'DNA_11',
 '21/35',
 'CD79b',
 'IgM',
 'DNA_12',
 'CD45',
 'ERTR7',
 'CD27',
 'DNA_13',
 'CD4',
 'PD-L1',
 'IgD',
 'DNA_14',
 'Ly6C',
 'CD24',
 'TCRb',
 'DNA_15',
 'BST2',
 'Ly6G',
 'B220',
 'DNA_16',
 '16/32',
 'CD3',
 'DNA_17',
 'CD38',
 'MHCII',
 'CD31',
 'DNA_18',
 'CD19',
 'DNA_19',
 'ICOS',
 'DNA_20',
 'DNA_21',
 'DNA_22',
 'DRAQ5']

In [11]:
selected_props = ["label", "eccentricity", "area", "equivalent_diameter",
                  "extent", "major_axis_length", "mean_intensity", "max_intensity"]

In [14]:
region_props = {}
for k, sm in segmentation_masks.items():
    ti = k[0]
    z = k[1]
    print(ti)
    img = read_tile_image(ti, channelNames, selected_channels, [z], home_path + "mouse_number_89_reg005_tiles")
    for ci, cn in enumerate(selected_channels):
        print(cn)
        rp = skimage.measure.regionprops_table(
            sm,
            img[ci, 0, :, :],
            properties = selected_props,
            cache = True
        )
        region_props[(k, ci)] = rp

reg005_X01_Y06
DNA_1
CD62L
Siglec-H
FOXP3
DNA_2
CD86
TCRgd
NKp46
DNA_3
CD25
CD69
CXCR5
DNA_4
Ki67
DNA_5
CD90
PNAd
TYRP1
DNA_6
CD11b
podoplanin
CD11c
DNA_7
Ter119
F4/80
CD106
DNA_8
CD5
H-2Db
DNA_9
CD71
H-2Kb
CD169
DNA_10
CD44
CD8a
PD1
DNA_11
21/35
CD79b
IgM
DNA_12
CD45
ERTR7
CD27
DNA_13
CD4
PD-L1
IgD
DNA_14
Ly6C
CD24
TCRb
DNA_15
BST2
Ly6G
B220
DNA_16
16/32
CD3
DNA_17
CD38
MHCII
CD31
DNA_18
CD19
DNA_19
ICOS
DNA_20
DNA_21
DNA_22
DRAQ5
reg005_X01_Y06
DNA_1
CD62L
Siglec-H
FOXP3
DNA_2
CD86
TCRgd
NKp46
DNA_3
CD25
CD69
CXCR5
DNA_4
Ki67
DNA_5
CD90
PNAd
TYRP1
DNA_6
CD11b
podoplanin
CD11c
DNA_7
Ter119
F4/80
CD106
DNA_8
CD5
H-2Db
DNA_9
CD71
H-2Kb
CD169
DNA_10
CD44
CD8a
PD1
DNA_11
21/35
CD79b
IgM
DNA_12
CD45
ERTR7
CD27
DNA_13
CD4
PD-L1
IgD
DNA_14
Ly6C
CD24
TCRb
DNA_15
BST2
Ly6G
B220
DNA_16
16/32
CD3
DNA_17
CD38
MHCII
CD31
DNA_18
CD19
DNA_19
ICOS
DNA_20
DNA_21
DNA_22
DRAQ5
reg005_X04_Y01
DNA_1
CD62L
Siglec-H
FOXP3
DNA_2
CD86
TCRgd
NKp46
DNA_3
CD25
CD69
CXCR5
DNA_4
Ki67
DNA_5
CD90
PNAd
TYRP1
DNA_6
CD

In [15]:
with open(shared_dir + "/region_props.pkl", "wb") as f:
    pickle.dump(region_props, f)

In [21]:
region_props[(('reg005_X01_Y06', 0), 4)]

{'label': array([16827]),
 'eccentricity': array([0.68151731]),
 'area': array([65]),
 'equivalent_diameter': array([9.09728368]),
 'extent': array([0.60185185]),
 'major_axis_length': array([11.17232716]),
 'mean_intensity': array([5418.76923077]),
 'max_intensity': array([8810.])}

In [27]:
region_props_df = defaultdict(list)
for ((ti, mi), z), p in region_props.items():
    for k, v in p.items():
        region_props_df[k].extend(v)
    region_props_df["marker"].extend([selected_channels[mi]]*len(p["label"]))
region_props_df = pd.DataFrame(region_props_df)

In [28]:
region_props_df

,label,eccentricity,area,equivalent_diameter,extent,major_axis_length,mean_intensity,max_intensity,marker
0,16827,0.681517,65,9.097284,0.601852,11.172327,1318.200000,3589.0,DNA_1
1,16827,0.681517,65,9.097284,0.601852,11.172327,2546.061538,3940.0,DNA_1
2,16827,0.681517,65,9.097284,0.601852,11.172327,3982.861538,12755.0,DNA_1
3,16827,0.681517,65,9.097284,0.601852,11.172327,12865.553846,47084.0,DNA_1
4,16827,0.681517,65,9.097284,0.601852,11.172327,5418.769231,8810.0,DNA_1
...,...,...,...,...,...,...,...,...,...
5221435,45649,0.828065,38,6.955796,0.527778,10.415979,5495.815789,7141.0,CD86
5221436,45650,0.719754,128,12.766153,0.914286,15.518582,4426.398438,5820.0,CD86
5221437,45651,0.402374,10,3.568248,0.625000,4.098780,4461.800000,4743.0,CD86
5221438,45652,0.841783,139,13.303394,0.817647,18.470042,3737.294964,6873.0,CD86


In [32]:
region_props_df.rename(columns={"label": "cell_ix"}, inplace=True)

In [37]:
region_props_df["CellID"] = [cell_ix_id_map[x] for x in region_props_df["cell_ix"]]

In [38]:
region_props_df

,cell_ix,eccentricity,area,equivalent_diameter,extent,major_axis_length,mean_intensity,max_intensity,marker,CellID
0,16827,0.681517,65,9.097284,0.601852,11.172327,1318.200000,3589.0,DNA_1,218-2873-0
1,16827,0.681517,65,9.097284,0.601852,11.172327,2546.061538,3940.0,DNA_1,218-2873-0
2,16827,0.681517,65,9.097284,0.601852,11.172327,3982.861538,12755.0,DNA_1,218-2873-0
3,16827,0.681517,65,9.097284,0.601852,11.172327,12865.553846,47084.0,DNA_1,218-2873-0
4,16827,0.681517,65,9.097284,0.601852,11.172327,5418.769231,8810.0,DNA_1,218-2873-0
...,...,...,...,...,...,...,...,...,...,...
5221435,45649,0.828065,38,6.955796,0.527778,10.415979,5495.815789,7141.0,CD86,4031-3196-5
5221436,45650,0.719754,128,12.766153,0.914286,15.518582,4426.398438,5820.0,CD86,4031-3208-5
5221437,45651,0.402374,10,3.568248,0.625000,4.098780,4461.800000,4743.0,CD86,4031-3217-5
5221438,45652,0.841783,139,13.303394,0.817647,18.470042,3737.294964,6873.0,CD86,4031-3226-5


In [41]:
region_props_df.to_csv(shared_dir + "/region_props.csv.gz", index=False)